In [1]:
import os  
import nest_asyncio
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

endpoint = os.getenv("ENDPOINT_URL")  
deployment = os.getenv("DEPLOYMENT_NAME")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("API_VERSION")

client = AzureOpenAIChatCompletionClient(
    azure_endpoint=endpoint,
    model=deployment,
    api_key=subscription_key,
    api_version=api_version
)

In [5]:
class HealthcareAgent:
    def __init__(self):
        """Inicializa os agentes e configura o ambiente."""
        print("[INFO] Inicializando os agentes de IA...")
        self.assistant_agent = AssistantAgent("MedicalAssistant", client)
        self.code_executor_agent = CodeExecutorAgent("CodeExecutor", client)

        # Caminhos para os arquivos de dados
        self.files = {
            "medical_record": "../data/medical_record.txt",
            "image_record": "../data/image_record.txt",
            "lab_record": "../data/lab_record.txt",
        }

    def identify_context(self, question: str):
        """Identifica a base de dados correta com base na pergunta do usuário."""
        print(f"[INFO] Identificando contexto para a pergunta: '{question}'")
        if "imagem" in question.lower():
            print("[INFO] Contexto identificado: Registros de Imagem.")
            return "image_record"
        elif "exame laboratorial" in question.lower() or "laboratório" in question.lower():
            print("[INFO] Contexto identificado: Registros Laboratoriais.")
            return "lab_record"
        else:
            print("[INFO] Contexto identificado: Registros Médicos.")
            return "medical_record"

    async def handle_medical_context(self, file_path, question):
        """Módulo de assistente para lidar com registros médicos."""
        print(f"[INFO] Lendo o arquivo médico: {file_path}")
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                medical_record = file.read()
        except FileNotFoundError:
            print(f"[ERROR] Arquivo não encontrado: {file_path}")
            return f"Arquivo não encontrado: {file_path}"

        print("[INFO] Criando o prompt para o agente 'MedicalAssistant'...")
        task_prompt = f"""
        Você é um assistente médico virtual especializado em entender e resumir histórias de pacientes. 
        Abaixo está a história médica completa do paciente:

        {medical_record}

        Com base na pergunta do usuário, resuma a história do paciente e forneça as informações solicitadas.
        Pergunta do usuário: "{question}"
        """
        print("[DEBUG] Prompt criado para o agente 'MedicalAssistant':")
        print(task_prompt[:500])  # Mostra os primeiros 500 caracteres do prompt

        print("[INFO] Enviando tarefa para o agente 'MedicalAssistant'...")
        response = await self.assistant_agent.run(task=task_prompt)
        print("[INFO] Resposta recebida do agente 'MedicalAssistant'.")
        return response

    async def handle_other_contexts(self, file_path, question):
        """Módulo para lidar com registros de imagens ou laboratoriais."""
        print(f"[INFO] Lendo o arquivo: {file_path}")
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                record = file.read()
        except FileNotFoundError:
            print(f"[ERROR] Arquivo não encontrado: {file_path}")
            return f"Arquivo não encontrado: {file_path}"

        print("[INFO] Criando o prompt para o agente 'MedicalAssistant'...")
        task_prompt = f"""
        Você é um assistente virtual especializado em fornecer informações detalhadas com base em diferentes fontes. 
        Abaixo está o conteúdo do cenário identificado:

        {record}

        Responda à pergunta do usuário com base no contexto fornecido:
        Pergunta do usuário: "{question}"
        """
        print("[DEBUG] Prompt criado para o agente 'MedicalAssistant':")
        print(task_prompt[:500])  # Mostra os primeiros 500 caracteres do prompt

        print("[INFO] Enviando tarefa para o agente 'MedicalAssistant'...")
        response = await self.assistant_agent.run(task=task_prompt)
        print("[INFO] Resposta recebida do agente 'MedicalAssistant'.")
        return response

    async def run(self, question):
        """Gerencia o fluxo principal com base na pergunta do usuário."""
        print(f"[INFO] Pergunta do usuário: {question}")
        context = self.identify_context(question)
        file_path = self.files.get(context)

        if not file_path or not os.path.exists(file_path):
            print(f"[ERROR] Arquivo {context}.txt não encontrado na pasta data.")
            return f"Arquivo {context}.txt não encontrado na pasta data."

        if context == "medical_record":
            print("[INFO] Chamando o módulo 'handle_medical_context' com o agente 'MedicalAssistant'...")
            return await self.handle_medical_context(file_path, question)
        else:
            print("[INFO] Chamando o módulo 'handle_other_contexts' com o agente 'MedicalAssistant'...")
            return await self.handle_other_contexts(file_path, question)

Digite sua pergunta: qual o ultimo exame laboratorial do paciente
[INFO] Inicializando os agentes de IA...
[INFO] Executando o fluxo principal...
[INFO] Pergunta do usuário: qual o ultimo exame laboratorial do paciente
[INFO] Identificando contexto para a pergunta: 'qual o ultimo exame laboratorial do paciente'
[INFO] Contexto identificado: Registros Laboratoriais.
[INFO] Chamando o módulo 'handle_other_contexts' com o agente 'MedicalAssistant'...
[INFO] Lendo o arquivo: ../data/lab_record.txt
[INFO] Criando o prompt para o agente 'MedicalAssistant'...
[DEBUG] Prompt criado para o agente 'MedicalAssistant':

        Você é um assistente virtual especializado em fornecer informações detalhadas com base em diferentes fontes. 
        Abaixo está o conteúdo do cenário identificado:

        Exame laboratorial mostrou plaquetas alteradas, outros resultados do hemograma estão normais

        Responda à pergunta do usuário com base no contexto fornecido:
        Pergunta do usuário: "qual o

In [ ]:
async def main():

    user_question = input("Digite sua pergunta: ")

    healthcare_agent = HealthcareAgent()

    print("[INFO] Executando o fluxo principal...")
    response = await healthcare_agent.run(user_question)
    print("\n[INFO] Resposta do assistente:")
    print(response)

nest_asyncio.apply()

await main()

In [26]:
import autogen_agentchat.agents

modules = dir(autogen_agentchat.agents)
print("Módulos, classes e funções disponíveis em autogen_agentchat.agents:")
for item in modules:
    print(item)

Módulos, classes e funções disponíveis em autogen_agentchat.agents:
AssistantAgent
BaseChatAgent
CodeExecutorAgent
SocietyOfMindAgent
UserProxyAgent
__all__
__builtins__
__cached__
__doc__
__file__
__loader__
__name__
__package__
__path__
__spec__
_assistant_agent
_base_chat_agent
_code_executor_agent
_society_of_mind_agent
_user_proxy_agent
